In [80]:
import tkinter as tk
from tkinter import filedialog
import os

In [85]:
 def Ask_Directory(self):
        dirName = filedialog.askdirectory()
        return dirName

class SampleApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        self.title('Give me Osu Songs')              # 창 제목 설정
        self.geometry("500x250+250+700")   # 창 크기 (너비x높이+x좌표+y좌표)
        self.resizable(False, False)   # 창 크기 조절 여부        
        
        self._frame = None
        self.switch_frame(StartPage)

    def switch_frame(self, frame_class):
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()

        


class StartPage(tk.Frame):  # 암기모드'

    def Select_Songs_Location(self):
        self.songs_location = Ask_Directory(self)
        self.songs_location_show.config(text="Songs Location : " + self.songs_location)

    def Select_Extract_Location(self):
        self.extract_location = Ask_Directory(self)
        self.extract_location_show.config(text="Extract Location : " + self.extract_location)

    def Extract_Start(self):
        print("시작")

    def __init__(self, master):  
        tk.Frame.__init__(self, master)
        tk.Frame.configure(self, width = 500, height = 250)

        #위에서부터 위젯 위치 순서대로 나열

        self.instruction = tk.Label(self, text = "안내: Songs 경로와 노래가 저장될 경로를 선택해주세요.") 
        self.instruction.place(x=25, y=15)
        
        self.songs_location_btn = tk.Button(self, text = 'Songs 경로 선택', anchor = 'center', command = self.Select_Songs_Location)
        self.songs_location_btn.place(x=25, y=50)    

        self.songs_location_show = tk.Label(self, text = "Song Location :") 
        self.songs_location_show.place(x=25, y=80)
        
        self.extract_location_btn = tk.Button(self, text = '추출될 경로 선택', anchor = 'center', command = self.Select_Extract_Location)
        self.extract_location_btn.place(x=25, y=110)  

        self.extract_location_show = tk.Label(self, text = "Extract Location :")
        self.extract_location_show.place(x=25, y=140)

        self.extract_start_btn = tk.Button(self, text = '추출 시작', anchor = 'center', command = self.Extract_Start)
        self.extract_start_btn.place(x=25, y=195)  


  



if __name__ == "__main__":
    app = SampleApp()
    app.mainloop()

    
   